In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *

import time

In [3]:
volume_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_volumes/'
if not os.path.exists(volume_dir):
    os.makedirs(volume_dir)

In [4]:
patch_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_data_patches/'

In [5]:
scoremaps_rootdir = '/oasis/projects/nsf/csd395/yuncong/CSHL_scoremaps_lossless'

In [6]:
# labels = ['BackG', '5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL']

labels = ['BackG', '5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL', 
                     '6N', 'Amb', 'R', 'Tz', 'Sol', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

label_dict = dict([(l,i) for i, l in enumerate(labels)])

# colors = (np.loadtxt(os.environ['REPO_DIR'] + '/visualization/100colors.txt')*255).astype(np.uint8)
# colors[label_dict['BackG']] = 255

In [7]:
downsample_factor = 16

section_thickness = 20 # in um
xy_pixel_distance_lossless = 0.46
xy_pixel_distance_tb = xy_pixel_distance_lossless * 32 # in um, thumbnail
# factor = section_thickness/xy_pixel_distance_lossless

xy_pixel_distance_downsampled = xy_pixel_distance_lossless * downsample_factor
z_xy_ratio_downsampled = section_thickness / xy_pixel_distance_downsampled

In [ ]:
##### reconstruct score volume ######

In [9]:
# stack = 'MD594'
# stack = 'MD593'

# for stack in ['MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602', 'MD594']:
# for stack in ['MD585']:
# for stack in ['MD590', 'MD591', 'MD595', 'MD598', 'MD602', 'MD594']:
for stack in ['MD592']:
    
    print stack

    section_roi_begin, section_roi_end = detect_bbox_range_lookup[stack]
    print section_roi_begin, section_roi_end

    dm = DataManager(stack=stack)

    roi_zmin = int(section_roi_begin*z_xy_ratio_downsampled)
    roi_zmax = int((section_roi_end+1)*z_xy_ratio_downsampled)-1
    print roi_zmin, roi_zmax


    volume_xmin, volume_ymin, volume_zmin = (np.inf, np.inf, np.inf)
    volume_xmax, volume_ymax, volume_zmax = (0,0,0)

    for l in labels:
    # for l in ['VLL']:

        if l == 'BackG': continue

        print l

        volume = np.zeros((dm.image_height/downsample_factor, 
                           dm.image_width/downsample_factor, 
                           int(section_number_lookup[stack]*z_xy_ratio_downsampled)), np.float32)
        print volume.shape

        for sec in range(section_roi_begin, section_roi_end+1):
            
#             t = time.time()

            scoremap_bp_filepath = scoremaps_rootdir + '/%(stack)s/%(slice)04d/%(stack)s_%(slice)04d_roi1_denseScoreMapLossless_%(label)s.bp' \
                % {'stack': stack, 'slice': sec, 'label': l}

            if not os.path.exists(scoremap_bp_filepath):
                sys.stderr.write('No scoremap for section %d\n' % sec)
                continue

#             print sec

            scoremap_roi = bp.unpack_ndarray_file(scoremap_bp_filepath)

            dataset = stack + '_' + '%04d'%sec + '_roi1'

            interpolation_xmin, interpolation_xmax, \
            interpolation_ymin, interpolation_ymax = np.loadtxt(os.path.join(scoremaps_rootdir, stack, '%04d'%sec,
                                                                             '%(dataset)s_denseScoreMapLossless_%(label)s_interpBox.txt' % \
                                            {'dataset': dataset, 'label': l})).astype(np.int)

            dense_scoremap_lossless = np.zeros((dm.image_height, dm.image_width), np.float32)

    #         dense_scoremap_lossless[interpolation_ymin:interpolation_ymax+1,
    #                                 interpolation_xmin:interpolation_xmax+1] = scoremap_roi.copy()
    #         scoremap = dense_scoremap_lossless[::downsample_factor, ::downsample_factor].copy()
    #         volume[:, :, int(sec*z_xy_ratio_downsampled):int((sec+1)*z_xy_ratio_downsampled)] = scoremap[..., np.newaxis].copy()

            dense_scoremap_lossless[interpolation_ymin:interpolation_ymax+1,
                                    interpolation_xmin:interpolation_xmax+1] = scoremap_roi
            scoremap = dense_scoremap_lossless[::downsample_factor, ::downsample_factor]
            volume[:, :, int(sec*z_xy_ratio_downsampled):int((sec+1)*z_xy_ratio_downsampled)] = scoremap[..., np.newaxis].copy()

            del scoremap, scoremap_roi, dense_scoremap_lossless

            
#             print time.time() - t
            
        ys, xs, zs = np.where(volume > 0)

        xmin = xs.min()
        xmax = xs.max()
        ymin = ys.min()
        ymax = ys.max()
        zmin = zs.min()
        zmax = zs.max()

        print xmin, xmax
        print ymin, ymax
        print zmin, zmax

        if xmin < volume_xmin:
            volume_xmin = xmin
        if ymin < volume_ymin:
            volume_ymin = ymin
        if zmin < volume_zmin:
            volume_zmin = zmin
        if xmax > volume_xmax:
            volume_xmax = xmax
        if ymax > volume_ymax:
            volume_ymax = ymax
        if zmax > volume_zmax:
            volume_zmax = zmax

        bp.pack_ndarray_file(volume[..., roi_zmin:roi_zmax+1],
                     os.path.join(volume_dir, 'volume_%(stack)s_scoreMap_%(label)s.bp' % {'stack': stack, 'label': l}))    

        del volume, ys, xs, zs

    print volume_xmin, volume_ymin, volume_zmin
    print volume_xmax, volume_ymax, volume_zmax

    with open(os.path.join(volume_dir, 'volume_%(stack)s_scoreMap_limits.txt' % {'stack': stack}), 'w') as f:
        f.write((' '.join(['%d']*6)+'\n') % (volume_xmin, volume_xmax, volume_ymin, volume_ymax, volume_zmin, volume_zmax))

MD592
147 319
399

No scoremap for section 149
No scoremap for section 249


 868
5N
(814, 1252, 1233)
88 920
259 727
399 868
7n

No scoremap for section 149
No scoremap for section 152
No scoremap for section 207
No scoremap for section 208
No scoremap for section 211
No scoremap for section 212
No scoremap for section 213
No scoremap for section 215
No scoremap for section 216
No scoremap for section 217
No scoremap for section 219
No scoremap for section 221
No scoremap for section 225
No scoremap for section 245
No scoremap for section 247



(814, 1252, 1233)
88 920
259 719
399 868
7N

...


(814, 1252, 1233)
88 920
259 732
399 868
AP

No scoremap for section 249
No scoremap for section 149
No scoremap for section 152
No scoremap for section 165
No scoremap for section 169
No scoremap for section 249
No scoremap for section 277
No scoremap for section 279
No scoremap for section 287
No scoremap for section 297
No scoremap for section 303
No scoremap for section 309



(814, 1252, 1233)
88 920
259 737
399 868
sp5

No scoremap for section 313
No scoremap for section 149



(814, 1252, 1233)
88 920
259 727
399 868
88 259 399
920 737 868
